In [130]:
import gc
import numpy as np
import pandas as pd

# 데이터경로

data_path = './competitive-data-science-predict-future-sales/'

sales_train = pd.read_csv(data_path + 'sales_train.csv')
shops = pd.read_csv(data_path + 'shops.csv')
items = pd.read_csv(data_path + 'items.csv')
item_categories = pd.read_csv(data_path + 'item_categories.csv')
test = pd.read_csv(data_path +'test.csv')
submission = pd.read_csv(data_path +'sample_submission.csv')

In [131]:
## 피처 엔지니어링 1 : 피처명 한글화와 데이터 다운캐스팅

sales_train = sales_train.rename(columns = {'date' : '날짜' ,
                                            'date_block_num' : '월ID',
                                            'shop_id' : '상점ID',
                                            'item_id' : '상품ID',
                                            'item_price' : '판매가',
                                            'item_cnt_day' : '판매량'})

shops = shops.rename(columns = {'shop_name' : '상점명' ,
                                'shop_id' : '상점ID'})

items = items.rename(columns = {'item_name': '상품명',
                                'item_id' : '상품ID',
                                'item_category_id' : '상품분류ID'})

item_categories = item_categories.rename(columns = {'item_category_name' : '상품분류명',
                                                    'item_category_id' : '상품분류ID'})

test = test.rename(columns = {'shop_id' : '상점ID',
                              'item_id' : '상품ID'})


In [132]:
def downcast(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col] , downcast= 'integer')
        else:
            df[col] = pd.to_numeric(df[col] , downcast ='float')

    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('{:.1f}% 압축됨'.format(100 * (start_mem - end_mem) / start_mem))

    return df

all_df = [sales_train , shops , items , item_categories ,test]

for df in all_df:
    df= downcast(df)

62.5% 압축됨
38.6% 압축됨
54.2% 압축됨
39.9% 압축됨
70.8% 압축됨


## 피처 엔지니어링 2 : 개별 데이터 피처 엔지니어링

In [133]:
# sales_train 이상치 제거 및 전처리

# sales_train의 판매가, 판매량 피처의 이상치를 제거한다.

# 판매가, 판매량이 음수라면 환불 건이거나 오류이다. 이에 판매가, 판매량이 음수인 데이터는 이상치로 간주한다.

# 판매가가 50,000이상인 데이터, 판매량이 1,000이상인 데이터도 이상치로 간주한다.

# 결론적으로 0~ 50,000 사이이고, 판매량이 0 ~ 1,000 사이인 데이터만 추출한다.

sales_train.head()

,날짜,월ID,상점ID,상품ID,판매가,판매량
0,02.01.2013,0,59,22154,999.000000,1
1,03.01.2013,0,25,2552,899.000000,1
2,05.01.2013,0,25,2552,899.000000,-1
3,06.01.2013,0,25,2554,1709.050049,1
4,15.01.2013,0,25,2555,1099.000000,1


In [134]:
# 판매가가 0 보다 큰 데이터 추출
sales_train = sales_train[sales_train['판매가'] >0]

# 판매가가 50000보다 작은 데이터 추출
sales_train = sales_train[sales_train['판매가']<50000]

# 판매량이 0보다 큰 데이터 추출

sales_train = sales_train[sales_train['판매량']> 0]

# 판매량이 1,000보다 작은 데이터 추출

sales_train = sales_train[sales_train['판매량']<1000]

In [135]:
print(shops['상점명'][0] , '||' , shops['상점명'][57])
print(shops['상점명'][1] , '||' , shops['상점명'][58])
print(shops['상점명'][10] , '||' , shops['상점명'][11])
print(shops['상점명'][39] , '||' , shops['상점명'][40])

!Якутск Орджоникидзе, 56 фран || Якутск Орджоникидзе, 56
!Якутск ТЦ "Центральный" фран || Якутск ТЦ "Центральный"
Жуковский ул. Чкалова 39м? || Жуковский ул. Чкалова 39м²
РостовНаДону ТРК "Мегацентр Горизонт" || РостовНаДону ТРК "Мегацентр Горизонт" Островной


In [136]:
# sales_train 과 test 데이터에서 상품ID 0은 57로 , 1은 58로, 10은 11로 , 39는 40으로 수정한다.
# 상점명은 놔두고 상품ID를 수정한다.

sales_train.loc[sales_train['상점ID'] == 0 , '상점ID'] = 57
sales_train.loc[sales_train['상점ID'] == 1 , '상점ID'] = 58
sales_train.loc[sales_train['상점ID'] == 10 , '상점ID'] = 11
sales_train.loc[sales_train['상점ID'] == 39 , '상점ID'] = 40

# test 데이터에서 상점ID 수정
test.loc[test['상점ID'] ==0 , '상점ID'] = 57
test.loc[test['상점ID'] ==1 , '상점ID'] = 58
test.loc[test['상점ID'] ==10 , '상점ID'] = 11
test.loc[test['상점ID'] ==39 , '상점ID'] = 40

In [137]:
# shops 파생 피처 생성 및 인코딩

# shops에도 상점명이 러시아어로 되어 있다.

# 상점명의 첫 단어가 도시라는 사실이 있다.

# 상점명을 이용해 도시 피처를 만들 수 있다.

shops.head()

,상점명,상점ID
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [138]:
shops['도시'] = shops['상점명'].apply(lambda x : x.split()[0])

shops['도시'].unique()

array(['!Якутск', 'Адыгея', 'Балашиха', 'Волжский', 'Вологда', 'Воронеж',
       'Выездная', 'Жуковский', 'Интернет-магазин', 'Казань', 'Калуга',
       'Коломна', 'Красноярск', 'Курск', 'Москва', 'Мытищи', 'Н.Новгород',
       'Новосибирск', 'Омск', 'РостовНаДону', 'СПб', 'Самара', 'Сергиев',
       'Сургут', 'Томск', 'Тюмень', 'Уфа', 'Химки', 'Цифровой', 'Чехов',
       'Якутск', 'Ярославль'], dtype=object)

In [139]:
shops.loc[shops['도시'] =='!Якутск' , '도시'] = 'Якутск'

# 도시명은 범주형 피처이다. 머신러닝 모델은 문자를 인식하지 못하므로 숫자로 바꿔야 한다.

# 레이블 인코딩을 적용한다.

# 레이블 인코딩의 단점은 서로 가까운 숫자를 비슷한 데이터로 판단하여 성능을 떨어뜨릴 수 있다. 트리 기반 모델을 사용할 땐 레이블 인코딩을 해도 큰 지장이 없다.

# 트리 기반 모델 특성상 분기를 반복하면서 피처 정보를 반영하므로 레이블 인코딩의 단점이 어느 정도 무마 된다.

In [140]:
from sklearn.preprocessing import LabelEncoder

# 레이블 인코더 생성
label_encoder = LabelEncoder()

# 도시 피처 레이블 인코딩
shops['도시'] = label_encoder.fit_transform(shops['도시'])

In [141]:
shops['도시']

0     29
1     29
2      0
3      1
4      2
5      3
6      4
7      4
8      4
9      5
10     6
11     6
12     7
13     8
14     8
15     9
16    10
17    11
18    11
19    12
20    13
21    13
22    13
23    13
24    13
25    13
26    13
27    13
28    13
29    13
30    13
31    13
32    13
33    14
34    15
35    15
36    16
37    16
38    17
39    18
40    18
41    18
42    19
43    19
44    20
45    20
46    21
47    22
48    23
49    24
50    24
51    24
52    25
53    25
54    26
55    27
56    28
57    29
58    29
59    30
Name: 도시, dtype: int32

In [142]:
# 상점명 피처를 활용해 도시 피처를 만들어 인코딩을 마쳤다. 이제 상점명 피처는 모델링에 더이상 필요가 없다.

shops = shops.drop('상점명' , axis = 1)

shops.head()

,상점ID,도시
0,0,29
1,1,29
2,2,0
3,3,1
4,4,2


## items 파생 피처 생성

In [143]:
#items를 활용하여 '첫 판매월' 피처를 구해본다.

# 상품명 피처 제거

items = items.drop(['상품명'], axis =1) # 상품명의 경우 상품ID와 일대일 매칭되어 있어서 제거해도 된다.

In [144]:
items.head()

,상품ID,상품분류ID
0,0,40
1,1,76
2,2,40
3,3,40
4,4,40


In [145]:
sales_train.head()

,날짜,월ID,상점ID,상품ID,판매가,판매량
0,02.01.2013,0,59,22154,999.000000,1
1,03.01.2013,0,25,2552,899.000000,1
3,06.01.2013,0,25,2554,1709.050049,1
4,15.01.2013,0,25,2555,1099.000000,1
5,10.01.2013,0,25,2564,349.000000,1


In [146]:
# 상품이 맨 처음 팔린 날을 피처 추가

items['첫 판매월'] = sales_train.groupby('상품ID').agg({'월ID' : 'min'})['월ID']

items

,상품ID,상품분류ID,첫 판매월
0,0,40,20.0
1,1,76,15.0
2,2,40,19.0
3,3,40,18.0
4,4,40,20.0
...,...,...,...
22165,22165,31,7.0
22166,22166,54,9.0
22167,22167,49,8.0
22168,22168,62,0.0


In [147]:
items[items['첫 판매월'].isna()] # 해당 상품이 한 번도 판매된 적이 없는 것

,상품ID,상품분류ID,첫 판매월
83,83,40,NaN
140,140,45,NaN
168,168,44,NaN
173,173,45,NaN
204,204,44,NaN
...,...,...,...
21974,21974,61,NaN
21975,21975,61,NaN
22022,22022,40,NaN
22035,22035,40,NaN


In [148]:
# 훈련 데이터는 2013년 1월 부터 2015년 10월 까지의 판매 내역이다.

# 월ID는 0부터 33이다.

# 테스트 데이터는 2015년 11월 판매 내역이다. 월ID는 34이다.

# 2013년 1월 부터 2015년 10월 까지 한 번도 팔리지 않은 상품이 있다면 그 상품이 처음 팔린 달을 2015년 11월이라고 가정해도 딘다.

# 첫 판매월 피처의 결측값을 34로 대체 해도 된다.

# 2015년 11월에도 안 팔릴 수 있다. 하지만 테스트 데이터에 없는 상품이면 아예 고려 대상이 되지 않는다.(테스트 데이터에도 없다는 말은 2015년 11월에도 안 팔렸다는 뜻이다.)

items['첫 판매월'] = items['첫 판매월'].fillna(34)

## item_categories 파생 피처 생성 및 인코딩



In [149]:
# item_cateories 에서 '대분류'라는 파생 피처를 만들고, 이를 인코딩해본다.

# item_categories는 상품분류명을 담고 있다. 상품분류명의 첫 단어가 범주 대분류이다.

item_categories['대분류'] = item_categories['상품분류명'].apply(lambda x: x.split()[0])

In [150]:
item_categories['대분류'].value_counts()

Игры          14
Книги         13
Подарки       12
Игровые        8
Аксессуары     7
Музыка         6
Программы      6
Кино           5
Карты          5
Чистые         2
Служебные      2
Доставка       1
Элементы       1
Билеты         1
PC             1
Name: 대분류, dtype: int64

## 피처 엔지니어링 3 : 데이터 조합 및 파생 피처 생성

In [151]:
# 데이터 조합

from itertools import product

train = []

# 월 ID, 상점 ID , 상품 ID 조합 생성

for i in sales_train['월ID'].unique():
    all_shop = sales_train.loc[sales_train['월ID'] ==i, '상점ID'].unique()
    all_item = sales_train.loc[sales_train['월ID']==i,  '상품ID'].unique()
    train.append(np.array(list(product([i] , all_shop, all_item))))

idx_features = ['월ID' , '상점ID' , '상품ID'] # 기준 피처
train = pd.DataFrame(np.vstack(train) , columns=idx_features)

train

,월ID,상점ID,상품ID
0,0,59,22154
1,0,59,2552
2,0,59,2554
3,0,59,2555
4,0,59,2564
...,...,...,...
10812763,33,21,7635
10812764,33,21,7638
10812765,33,21,7640
10812766,33,21,7632


In [152]:
# 파생 피처 생성

group = sales_train.groupby(idx_features).agg({'판매량' : 'sum', '판매가' : 'mean'})

group = group.reset_index()
group = group.rename(columns = {'판매량' : '월간 판매량' , '판매가' : '평균 판매가'})

train = train.merge(group , on = idx_features , how= 'left')

train.head()

,월ID,상점ID,상품ID,월간 판매량,평균 판매가
0,0,59,22154,1.0,999.0
1,0,59,2552,NaN,NaN
2,0,59,2554,NaN,NaN
3,0,59,2555,NaN,NaN
4,0,59,2564,NaN,NaN


In [153]:
import gc
# group 변수 가비지 컬렉션

del group
gc.collect()

52

In [154]:
# 상품 판매건수 피처 추가
group = sales_train.groupby(idx_features).agg({'판매량' : 'count'})
group = group.reset_index()
group = group.rename(columns = {'판매량' : '판매건수'})


group.head()

,월ID,상점ID,상품ID,판매건수
0,0,2,27,1
1,0,2,33,1
2,0,2,317,1
3,0,2,438,1
4,0,2,471,2


In [155]:
train = train.merge(group , on=idx_features , how = 'left')

# 가비지 컬렉션

del group, sales_train
gc.collect()

train.head()

,월ID,상점ID,상품ID,월간 판매량,평균 판매가,판매건수
0,0,59,22154,1.0,999.0,1.0
1,0,59,2552,NaN,NaN,NaN
2,0,59,2554,NaN,NaN,NaN
3,0,59,2555,NaN,NaN,NaN
4,0,59,2564,NaN,NaN,NaN


## 피처 엔지니어링 4 : 데이터 합치기

In [156]:
# 테스트 데이터 월ID를 34로 설정
test['월ID'] = 34

# train과 test 이어 붙이기
all_data = pd.concat([train , test.drop('ID' , axis =1)],
                     ignore_index= True,
                     keys= idx_features)

# 결측값을 0으로 대체
all_data = all_data.fillna(0)

all_data.head()

# train에 test를 이어붙여 all_data를 만들었다.

,월ID,상점ID,상품ID,월간 판매량,평균 판매가,판매건수
0,0,59,22154,1.0,999.0,1.0
1,0,59,2552,0.0,0.0,0.0
2,0,59,2554,0.0,0.0,0.0
3,0,59,2555,0.0,0.0,0.0
4,0,59,2564,0.0,0.0,0.0


In [157]:
# 나머지 데이터 병합
all_data = all_data.merge(shops, on ='상점ID' , how = 'left')
all_data = all_data.merge(items, on ='상품ID' , how = 'left')
all_data = all_data.merge(item_categories , on='상품분류ID' , how ='left')

# 데이터 다운캐스팅
all_data = downcast(all_data)

44.9% 압축됨


In [158]:
# 가비지 컬렉션

del shops, items, item_categories
gc.collect()

45

## 피처 엔지니어링 5 : 시차 피처 생성

In [159]:
# 시차 피처란 과거 시점에 관한 피처로 , 성능 향상에 도움되는 경우가 많아서 시계열 문제에서 자주 만드는 파생 피처이다.

# 시차 피처를 만들기 위해선 기준으로 삼을 피처를 먼저 정해야 한다.

# 타깃값과 관련된 '월간 평균 판매량'이 좋다.

# 기준 피처별 월간 평균 판매량 피처를 만들어야 한다.

In [160]:
idx_features

['월ID', '상점ID', '상품ID']

In [161]:
# 기준 피처별 월간 판매량 파생 피처 생성

# 파라미터

# df : DataFrame , mean_features : 새로 만든 월간 평균 판매량 파생 피처명을 저장하는 리스트 , idx_features : 기준 피처

# 기준 피처의 첫 번째 요소는 반드시 '월ID'여야 한다. '월간' 평균 판매량 파생 피처를 만들 것이다.


def add_mean_features(df , mean_features , idx_features):
    # 기준 피처 확인
    assert (idx_features[0] == '월ID') and \
        len(idx_features) in [2,3]

    # 기준 피처의 첫 번째 요소가 '월ID'가 맞는지 , 기준 피처 개수가 2개 혹은 3개인지 확인한다. 아닐경우 오류를 발생시킨다.

    # 파생 피처명 설정
    if len(idx_features) ==2 : # 기준 피처가 2개일 때와 3개일 때로 나누어 설정한다.
        feature_name = idx_features[1] + '별 평균 판매량'
    else:
        feature_name = idx_features[1] + ' ' + idx_features[2] + '별 평균 판매량'

    # 기준 피처를 토대로 그룹화해 월간 평균 판매량 구하기
    group = df.groupby(idx_features).agg({'월간 판매량' : 'mean'})
    group = group.reset_index()
    group = group.rename(columns = {'월간 판매량' : feature_name})

    # df와 group 병합
    df = df.merge(group , on=idx_features , how='left')

    # 데이터 다운캐스팅
    df = downcast(df, verbose=False)
    # verbose = False 를 하면 몇 %를 압축했습니다. 문구가 뜨지 않는다.

    # 새로 만든 feature_name 피처명을 mean_features 리스트에 추가
    mean_features.append(feature_name)

    # 가비지 컬렉션
    del group
    gc.collect()

    return df,mean_features

In [162]:
# 그룹화 기준 피처 중 '상품ID'가 포함된 파생 피처명을 담을 리스트
item_mean_features = []

# ['월ID' , '상품ID'] 로 그룹화한 월간 평균 판매량 파생 피처 생성
all_data , item_mean_features = add_mean_features(df = all_data,
                                                  mean_features=item_mean_features,
                                                  idx_features=['월ID' , '상품ID'])

# ['월ID' , '상품ID', '도시']로 그룹화한 월간 평균 판매량 파생 피처 생성
all_data , item_mean_features = add_mean_features(df = all_data ,
                                                  mean_features= item_mean_features,
                                                  idx_features=['월ID' , '상품ID' ,'도시'])

all_data.head()

,월ID,상점ID,상품ID,월간 판매량,평균 판매가,판매건수,도시,상품분류ID,첫 판매월,상품분류명,대분류,상품ID별 평균 판매량,상품ID 도시별 평균 판매량
0,0,59,22154,1,999.0,1,30,37,0,Кино - Blu-Ray,Кино,0.400000,1.0
1,0,59,2552,0,0.0,0,30,58,0,Музыка - Винил,Музыка,0.022222,0.0
2,0,59,2554,0,0.0,0,30,58,0,Музыка - Винил,Музыка,0.022222,0.0
3,0,59,2555,0,0.0,0,30,56,0,Музыка - CD фирменного производства,Музыка,0.044444,0.0
4,0,59,2564,0,0.0,0,30,59,0,Музыка - Музыкальное видео,Музыка,0.111111,0.0


In [163]:
item_mean_features

['상품ID별 평균 판매량', '상품ID 도시별 평균 판매량']

In [164]:
# 그룹화 기준 피처 중 '상점ID'가 포함된 파생 피처명을 담을 리스트
shop_mean_features = []

# ['월ID' , '상점ID' , '상품분류ID'] 로 그룹화한 월간 평균 판매량 파생 피처 생성

all_data , shop_mean_features = add_mean_features(df = all_data,
                                                  mean_features= shop_mean_features,
                                                  idx_features=['월ID' , '상점ID', '상품분류ID'])

# 상점ID를 포함한 파생 피처명은 shop_mean_features 리스트에 따로 담았다.

# 상품ID를 포함한 파생 피처명은 item_mean_features 리스트에 담았다.

In [165]:
shop_mean_features

['상점ID 상품분류ID별 평균 판매량']

In [166]:
# 시차 피처 생성 원리 및 함수 구현

# 기준 피처별 월간 평균 판매량 피처는

# 1. 상품ID별 평균 판매량

# 2. 상품ID + 도시 별 평균판매량

# 3. 상점ID + 상품분류ID 별 평균 판매량


# 시차 피처는 현시점 데이터에 과거 시점 데이터를 추가한다는 개념이다. 과거 시점 데이터는 향후 판매량 예측에 유용하기 때문에 사용한다.


def add_lag_features(df , lag_features_to_clip , idx_features , lag_feature , nlags = 3 , clip = False):
    # 시차 피처 생성에 필요한 DataFrame 부분만 복사
    df_temp = df[idx_features + [lag_feature].copy()]

    # 시차 피처 생성
    for i in range(1, nlags+1):
        # 시차 피처명
        lag_feature_name = lag_feature +'_시차' + str(i)
        # df_temp 열 이름 설정
        df_temp.columns = idx_features + [lag_feature_name]

        # df_temp의 date_block_num 피처에 1 더하기
        df_temp['월ID'] +=1
        # idx_feature를 기준으로 df와 df_temp 병합하기
        df = df.merge(df_temp.drop_duplicates(),
                      on = idx_features,
                      how = 'left')

        # 결측값 0으로 대체
        df[lag_feature_name] = df[lag_feature_name].fillna(0)

        # 0~20 사이로 제한할 시차 피처명을 lag_features_to_clip에 추가
        if clip:
            lag_features_to_clip.append(lag_feature_name)

        # 데이터 다운캐스팅
        df = downcast(df, False)

        # 가비지 컬렉션
        del df_temp
        gc.collect()

        return df,lag_features_to_clip

## 시차 피처 생성 1 : 월간 판매량

In [167]:
# 기준피처는 '월ID' , '상점ID' , '상품ID'로 하여 월간 판매량의 세 달치 시차 피처를 만들어본다.

# clip = True를 전달해 세 달치 시차 피처를 lag_features_to_clip 리스트에 저장해둔다.

# 월간 판매량은 타깃값이므로 0~20 사이로 제한해야 한다.

lag_features_to_clip = [] # 0~20 사이로 제한할 시차 피처명을 담을 리스트
idx_features = ['월ID' , '상점ID' , '상품ID'] # 기준 피처

# idx_features 를 기준으로 월간 판매량의 세달치 시차 피처 생성
all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                  lag_features_to_clip = lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='월간 판매량',
                                                  nlags= 3,
                                                  clip = True) # 값을 0~20 사이로 제한

all_data.head().T

<ipython-input-166-4d22d4386aa0>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['월ID'] +=1


,0,1,2,3,4
월ID,0,0,0,0,0
상점ID,59,59,59,59,59
상품ID,22154,2552,2554,2555,2564
월간 판매량,1,0,0,0,0
평균 판매가,999.0,0.0,0.0,0.0,0.0
판매건수,1,0,0,0,0
도시,30,30,30,30,30
상품분류ID,37,58,58,56,59
첫 판매월,0,0,0,0,0
상품분류명,Кино - Blu-Ray,Музыка - Винил,Музыка - Винил,Музыка - CD фирменного производства,Музыка - Музыкальное видео


In [168]:
lag_features_to_clip

['월간 판매량_시차1']

## 시차 피처 생성 2: 판매건수 , 평균 판매가

In [169]:
# idx_features를 기준으로 판매건수 피처의 세 달치 시차 피처 생성
all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                  lag_features_to_clip = lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='판매건수',
                                                  nlags= 3)


# idx_features를 기준으로 평균 판매가 피처의 세 달치 시차 피처 생성
all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                  lag_features_to_clip = lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='평균 판매가',
                                                  nlags= 3)



<ipython-input-166-4d22d4386aa0>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['월ID'] +=1
<ipython-input-166-4d22d4386aa0>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['월ID'] +=1


## 시차 피처 생성 3: 평균판매량

In [170]:
# idx_features를 기준으로 item_mean_Features 요소별 시차 피처 생성
for item_mean_features in item_mean_features:
    all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                      lag_features_to_clip = lag_features_to_clip,
                                                      idx_features=idx_features,
                                                      lag_feature=item_mean_features,
                                                      nlags= 3,
                                                      clip = True) # 값을 0~20 사이로 제한

# item_mean_features 피처 제거

all_data = all_data.drop(item_mean_features, axis =1 )

all_data

<ipython-input-166-4d22d4386aa0>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['월ID'] +=1
<ipython-input-166-4d22d4386aa0>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['월ID'] +=1


,월ID,상점ID,상품ID,월간 판매량,평균 판매가,판매건수,도시,상품분류ID,첫 판매월,상품분류명,대분류,상품ID별 평균 판매량,상점ID 상품분류ID별 평균 판매량,월간 판매량_시차1,판매건수_시차1,평균 판매가_시차1,상품ID별 평균 판매량_시차1,상품ID 도시별 평균 판매량_시차1
0,0,59,22154,1,999.0,1,30,37,0,Кино - Blu-Ray,Кино,0.400000,0.088496,0,0,0.0,0.000000,0.0
1,0,59,2552,0,0.0,0,30,58,0,Музыка - Винил,Музыка,0.022222,0.000000,0,0,0.0,0.000000,0.0
2,0,59,2554,0,0.0,0,30,58,0,Музыка - Винил,Музыка,0.022222,0.000000,0,0,0.0,0.000000,0.0
3,0,59,2555,0,0.0,0,30,56,0,Музыка - CD фирменного производства,Музыка,0.044444,0.008333,0,0,0.0,0.000000,0.0
4,0,59,2564,0,0.0,0,30,59,0,Музыка - Музыкальное видео,Музыка,0.111111,0.011976,0,0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11026963,34,45,18454,0,0.0,0,20,55,23,Музыка - CD локального производства,Музыка,0.000000,0.000000,1,1,99.0,0.045455,0.5
11026964,34,45,16188,0,0.0,0,20,64,32,Подарки - Настольные игры,Подарки,0.000000,0.000000,0,0,0.0,0.022727,0.0
11026965,34,45,15757,0,0.0,0,20,55,0,Музыка - CD локального производства,Музыка,0.000000,0.000000,0,0,0.0,0.113636,0.0
11026966,34,45,19648,0,0.0,0,20,40,23,Кино - DVD,Кино,0.000000,0.000000,0,0,0.0,0.045455,0.0


In [171]:
#['월ID' , '상점ID' , '상품분류ID']를 기준으로

# shop_mean_features 요소별 시차 피처 생성

for shop_mean_feature in shop_mean_features:
    all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                      lag_features_to_clip = lag_features_to_clip,
                                                      idx_features=['월ID' , '상점ID' , '상품분류ID'],
                                                      lag_feature=shop_mean_feature,
                                                      nlags= 3,
                                                      clip = True) # 값을 0~20 사이로 제한

all_data = all_data.drop(shop_mean_features, axis =1 )

all_data

<ipython-input-166-4d22d4386aa0>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['월ID'] +=1


,월ID,상점ID,상품ID,월간 판매량,평균 판매가,판매건수,도시,상품분류ID,첫 판매월,상품분류명,대분류,상품ID별 평균 판매량,월간 판매량_시차1,판매건수_시차1,평균 판매가_시차1,상품ID별 평균 판매량_시차1,상품ID 도시별 평균 판매량_시차1,상점ID 상품분류ID별 평균 판매량_시차1
0,0,59,22154,1,999.0,1,30,37,0,Кино - Blu-Ray,Кино,0.400000,0,0,0.0,0.000000,0.0,0.000000
1,0,59,2552,0,0.0,0,30,58,0,Музыка - Винил,Музыка,0.022222,0,0,0.0,0.000000,0.0,0.000000
2,0,59,2554,0,0.0,0,30,58,0,Музыка - Винил,Музыка,0.022222,0,0,0.0,0.000000,0.0,0.000000
3,0,59,2555,0,0.0,0,30,56,0,Музыка - CD фирменного производства,Музыка,0.044444,0,0,0.0,0.000000,0.0,0.000000
4,0,59,2564,0,0.0,0,30,59,0,Музыка - Музыкальное видео,Музыка,0.111111,0,0,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11026963,34,45,18454,0,0.0,0,20,55,23,Музыка - CD локального производства,Музыка,0.000000,1,1,99.0,0.045455,0.5,0.126866
11026964,34,45,16188,0,0.0,0,20,64,32,Подарки - Настольные игры,Подарки,0.000000,0,0,0.0,0.022727,0.0,0.094488
11026965,34,45,15757,0,0.0,0,20,55,0,Музыка - CD локального производства,Музыка,0.000000,0,0,0.0,0.113636,0.0,0.126866
11026966,34,45,19648,0,0.0,0,20,40,23,Кино - DVD,Кино,0.000000,0,0,0.0,0.045455,0.0,0.083756


In [172]:
## 시차 피처 생성 마무리 : 결측값 처리

# 월ID가 3 미만인 데이터 제거

all_data = all_data.drop(all_data[all_data['월ID'] < 3].index)

all_data

,월ID,상점ID,상품ID,월간 판매량,평균 판매가,판매건수,도시,상품분류ID,첫 판매월,상품분류명,대분류,상품ID별 평균 판매량,월간 판매량_시차1,판매건수_시차1,평균 판매가_시차1,상품ID별 평균 판매량_시차1,상품ID 도시별 평균 판매량_시차1,상점ID 상품분류ID별 평균 판매량_시차1
1122386,3,25,8092,3,149.0,3,13,40,0,Кино - DVD,Кино,0.622222,1,1,149.0,0.586957,0.666667,1.035039
1122387,3,25,7850,3,199.0,3,13,30,0,Игры PC - Стандартные издания,Игры,1.577778,5,4,199.0,1.869565,2.916667,3.554645
1122388,3,25,8051,3,30.0,2,13,66,0,"Подарки - Открытки, наклейки",Подарки,0.111111,0,0,0.0,0.043478,0.000000,1.796610
1122389,3,25,8088,1,199.0,1,13,55,0,Музыка - CD локального производства,Музыка,0.088889,1,1,199.0,0.130435,0.166667,0.569322
1122390,3,25,8089,1,199.0,1,13,55,0,Музыка - CD локального производства,Музыка,0.466667,0,0,0.0,0.478261,0.666667,0.569322
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11026963,34,45,18454,0,0.0,0,20,55,23,Музыка - CD локального производства,Музыка,0.000000,1,1,99.0,0.045455,0.500000,0.126866
11026964,34,45,16188,0,0.0,0,20,64,32,Подарки - Настольные игры,Подарки,0.000000,0,0,0.0,0.022727,0.000000,0.094488
11026965,34,45,15757,0,0.0,0,20,55,0,Музыка - CD локального производства,Музыка,0.000000,0,0,0.0,0.113636,0.000000,0.126866
11026966,34,45,19648,0,0.0,0,20,40,23,Кино - DVD,Кино,0.000000,0,0,0.0,0.045455,0.000000,0.083756


In [173]:
## 피처 엔지니어링 7 : 기타 피처 엔지니어링

# 기타 피처

# 월간 판매량 시차 피처들의 평균

all_data['월간 판매량 시차평균'] = all_data[['월간 판매량_시차1',
                                    '월간 판매량_시차2',
                                    '월간 판매량_시차3']].mean(axis=1)


#0~20 사이로 값 제한

all_data[lag_features_to_clip + ['월간 판매량' , '월간 판매량 시차평균']] = \
all_data[lag_features_to_clip + ['월간 판매량' , '월간 판매량 시차평균']].clip(0,20)




KeyError: "['월간 판매량_시차3', '월간 판매량_시차2'] not in index"